## Import packages

In [1]:
import pandas as pd
import numpy as np

## Load data

In [2]:
activity_data = pd.read_csv('../Raw Data/ACTIVITY.csv')

In [3]:
activity_data.shape

(66, 6)

In [4]:
activity_data.head()

,date,lastSyncTime,steps,distance,runDistance,calories
0,2019-04-12,1555128688,0,0,0,0
1,2019-04-13,1555256891,0,0,0,0
2,2019-04-14,1555302659,843,576,75,25
3,2019-04-15,1555384520,6158,4287,430,167
4,2019-04-16,1555473482,17791,13677,462,561


In [5]:
sleep_data = pd.read_csv('../Raw Data/SLEEP.csv')

In [6]:
sleep_data.shape

(66, 7)

In [7]:
sleep_data.head()

,date,lastSyncTime,deepSleepTime,shallowSleepTime,wakeTime,start,stop
0,2019-04-12,1555128688,0,0,0,1554930000,1554930000
1,2019-04-13,1555256891,0,0,0,1555016400,1555016400
2,2019-04-14,1555302659,0,0,0,1555102800,1555102800
3,2019-04-15,1555384520,156,287,0,1555273620,1555300200
4,2019-04-16,1555473482,203,307,2,1555353480,1555384200


In [8]:
huawei_data = pd.read_excel('../Raw Data/HUAWEI.xlsx', sheet_name='Sheet1')

In [9]:
huawei_data.shape

(66, 4)

In [10]:
huawei_data.head()

,Date,Steps,Distance,Calories
0,2019-04-12,3672,2.78,119
1,2019-04-13,5223,3.95,170
2,2019-04-14,1682,1.27,55
3,2019-04-15,2713,2.05,92
4,2019-04-16,16007,12.10,545


## Data analysis